In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
the_data = pd.read_csv('../input/water-potability/water_potability.csv')
the_data.info()

# Missing values (null values)
# Dtype of each columns contained in DataFrame
# Number of columns for selection of input and outcome features selection

In [ ]:
the_data.isnull().sum() # Total numbers of missing values in each column

# Data Cleaning Step

In [ ]:
the_data_2 = the_data.copy()
null_columns = [i for i in the_data_2.columns if the_data_2[i].isnull().any()]
print (null_columns)


from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors = 2,weights = 'uniform')
the_data_2[null_columns] = imputer.fit_transform(the_data_2[null_columns])

print(the_data_2.head())
# Imputation removes column names

the_data_2.isnull().sum()
# Now there is no null values

# Features Selection

In [ ]:
X = the_data_2.drop(['Potability'],axis = 1) # drop the column nammed (Potability) which is going to use as a labeled features
                                                    # leftover columns after drop will be used as training features

print (X.info())

y = the_data_2.Potability # This column will be used as a labeld features

# Build Model

In [ ]:
# As the goal is predicting whether the water is drinkable or not based on the lab parameters 
# The problem is classification problem and it is better to use supervised classification models

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# model = RandomForestClassifier(n_estimators = 100, random_state = 4) # uncomment for single model without cross validation

pipeline = Pipeline(steps=[('model',RandomForestClassifier(n_estimators = 100,random_state = 5))])

score = -1 * cross_val_score(pipeline,X,y,cv = 5,scoring = 'neg_mean_absolute_error')

print ("MAE scores:\n" ,score)
print (score.mean())

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.8,test_size = 0.2,random_state = 12)

# Model 2
model2 = RandomForestClassifier(n_estimators = 1000,criterion = "gini",)
model2.fit(X_train,y_train)
prediction2 = model2.predict(X_test)
print (mean_absolute_error(prediction2,y_test))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model3 = GradientBoostingClassifier(n_estimators = 1000,learning_rate = 0.01,criterion = 'mae')

model3.fit(X_train,y_train)
prediction3 = model3.predict(X_test)
print (mean_absolute_error(prediction3,y_test))

# Restarting everything from the base as I forget to split before moving on others

In [ ]:
data_3 = the_data.copy()

print (data_3.info())

X = data_3.drop(['Potability'],axis = 1)
y = data_3.Potability

X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.8,test_size = 0.2,random_state = 69)
X_train.shape
y_train.shape

In [ ]:
from sklearn.impute import SimpleImputer


imputer = SimpleImputer()
X_train_impute = pd.DataFrame(imputer.fit_transform(X_train))
X_test_impute = pd.DataFrame(imputer.transform(X_test))

In [ ]:
X_train_impute.columns  = X_train.columns
X_test_impute.columns = X_test.columns

*** Naive Bayes Classifier Model ***

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

prediction = gnb.fit(X_train_impute, y_train).predict(X_test_impute)

print("Number of mislabeled points out of a total %d points : %d"
        % (X_test_impute.shape[0], (y_test != prediction).sum()))


In [ ]:
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators = 100,learning_rate = 0.01,criterion = 'mae')

model.fit(X_train_impute,y_train)
prediction = model.predict(X_test_impute)
print (mean_absolute_error(prediction,y_test))

In [ ]:
print("Number of mislabeled points out of a total %d points : %d"
        % (X_test_impute.shape[0], (y_test != prediction).sum()))